# General tool for creating Muscle Force Direction setup xml file

> Alex Woodall<br>
> Auckland Bioengineering Institute<br>
> Auckland, New Zealand

This notebook will create a muscle analysis setup xml file by using xml parsing. This tool is 3rd party and therefore is not supported by the OpenSim API.

__Assuming that the motion capture data was pre-processed in Nexus to remove gaps and duplicate markers.__

This notebook is better used as a function call where the input variables have been created prior to calling.

Start by importing the necessary libraries

In [1]:
import numpy as np
from xml.dom import minidom
import sys

sys.path.insert(1, r'Functions')  # add to path
from xml_shorten import xml_shorten

## Requirements

To generate the muscle force direction setup xml file, we will need six items:
- __force_filename__: the full filename for the template muscle force direction setup xml file
- __trial__: the trial name, e.g., "12_Mar_ss_12ms_01"
- __model__: the model name, e.g., "AB08"
- __directory__: the output directory name
- __time__range__: the start and end times of the trial
- __cut_off_freq__: the low pass cut-off frequency (Hz)

In [6]:
# Establish input variables example
force_filename = 'C:\\Users\\alexw\\Dropbox\\ABI\\Level_8_Lab\\OpenSim Tools\\ProcessingTrialDataFromVicon\\xmlTemplates\\MuscleForceDirectionSetup.xml'
trial = '_12Mar_ss_12ms_01'
model = 'AB08'
directory = 'C:\\Users\\alexw\\Dropbox\\ABI\\Level_8_Lab\\OpenSim Tools\\ProcessingTrialDataFromVicon\\Output'
time__range = [15.3, 30.455]
cut_off_freq = 10

## Establish output, time, and filtering variables

In [7]:
doc_node = minidom.parse(force_filename)

''' Get Hierarchy Access '''
analyze_tool = doc_node.getElementsByTagName("AnalyzeTool")
analyze_tool_child = analyze_tool.item(0)

res_directory = analyze_tool_child.getElementsByTagName("results_directory")
res_directory_child = res_directory.item(0)

model_file = analyze_tool_child.getElementsByTagName("model_file")
model_file_child = model_file.item(0)

initial_time = analyze_tool_child.getElementsByTagName("initial_time")
initial_time_child = initial_time.item(0)

final_time = analyze_tool_child.getElementsByTagName("final_time")
final_time_child = final_time.item(0)

ex_loads_file = analyze_tool_child.getElementsByTagName("external_loads_file")
ex_loads_file_child = ex_loads_file.item(0)

coords_file = analyze_tool_child.getElementsByTagName("coordinates_file")
coords_file_child = coords_file.item(0)

analysis_set = analyze_tool_child.getElementsByTagName("AnalysisSet")
analysis_set_child = analysis_set.item(0)

objects_set = analysis_set_child.getElementsByTagName("objects")
objects_set_child = objects_set.item(0)

filter_frequency = analyze_tool_child.getElementsByTagName("lowpass_cutoff_frequency_for_coordinates")
filter_frequency_child = filter_frequency.item(0)

## Muscle force direction settings

In [8]:
muscle_force_direction = objects_set_child.getElementsByTagName("MuscleForceDirection")
muscle_force_direction_child = muscle_force_direction.item(0)

muscle_force_direction_start_time = muscle_force_direction_child.getElementsByTagName("start_time")
muscle_force_direction_start_time_child = muscle_force_direction_start_time.item(0)

muscle_force_direction_end_time = muscle_force_direction_child.getElementsByTagName("end_time")
muscle_force_direction_end_time_child = muscle_force_direction_end_time.item(0)

## Set new directory, filenames, and number outputs

In [9]:
new_analyze_tool_name = model + trial
analyze_tool_child.setAttribute('name', new_analyze_tool_name)

# Local directory
res_directory_child.firstChild.data = directory + "\\" + model + "\\" + trial

# OpenSim model name
model_file_name = directory + "\\" + model + "\\" + model + ".osim"
model_file_child.firstChild.data = model_file_name

external_loads_file = directory + "\\" + model + "\\" + trial + "\\" + trial + 'ExternalLoads.xml'
ex_loads_file_child.firstChild.data = external_loads_file

coordsfile = directory + "\\" + model + "\\" + trial + "\\" + trial + 'IKResults.mot'
coords_file_child.firstChild.data = coordsfile

filter_frequency_child.firstChild.data = str(cut_off_freq)

# Set start and end time for all tools
starttime = str(time__range[0])
endtime = str(time__range[-1])

initial_time_child.firstChild.data = starttime
final_time_child.firstChild.data = endtime

muscle_force_direction_start_time_child.firstChild.data = starttime
muscle_force_direction_end_time_child.firstChild.data = endtime

## Write to an XML setup file

In [12]:
xml_setup_path = directory + "\\" + model + "\\" + trial + "\\" + trial + "MuscleForceDirectionSetup.xml"

with open(xml_setup_path, 'w') as xml_file:
    doc_node.writexml(xml_file, addindent='\t', newl='\n', encoding='UTF-8')
    
# Using minidom will often create unneeded white space
xml_shorten(xml_setup_path)